<a href="https://colab.research.google.com/github/lalith0203/StockPrice_Prediction/blob/main/StockPrice_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
df=pd.read_csv("NSE-TATAGLOBAL.csv")
df.head()

In [ ]:
df["Date"]=pd.to_datetime(df.Date,format="%Y-%m-%d")
df.index=df['Date']
df.head()

In [ ]:
fig, axes = plt.subplots(nrows=2,ncols=2, figsize=(10, 7), squeeze=False)
fig.tight_layout(pad=5)
plt.style.use('fivethirtyeight')
axes[0][0].plot(df["Open"],linewidth=0.8)
axes[0][0].plot(df["Close"],linewidth=0.8)
axes[0][0].set_ylabel("Projections")
axes[0][0].set_xlabel("Date")
axes[0][0].set_title('Open Vs Close')
axes[0][0].legend(['Open', 'Close'], loc='upper left')
axes[0][1].plot(df["High"],linewidth=0.8)
axes[0][1].plot(df["Low"],linewidth=0.8)
axes[0][1].plot(df["Last"],linewidth=0.8)
axes[0][1].set_ylabel("Projections")
axes[0][1].set_xlabel("Date")
axes[0][1].set_title('High Vs Low Vs Last')
axes[0][1].legend(['High', 'Low', 'Last'], loc='upper left')
axes[1][0].plot(df["Total Trade Quantity"],linewidth=0.8)
axes[1][0].set_ylabel("Projections")
axes[1][0].set_xlabel("Date")
axes[1][0].set_title('Total Trade Quantity')
axes[1][0].legend(['Total Trade Quantity'], loc='upper left')
axes[1][1].plot(df["Turnover (Lacs)"],linewidth=0.8)
axes[1][1].set_ylabel("Projections")
axes[1][1].set_xlabel("Date")
axes[1][1].set_title('Turnover (Lacs)')
axes[1][1].legend(['Turnover (Lacs)'], loc='upper left')

In [ ]:
data=df.sort_index(ascending=True,axis=0)
new_dataset=pd.DataFrame(index=range(0,len(df)),columns=['Date','Close'])
for i in range(0,len(data)):
 new_dataset["Date"][i]=data['Date'][i]
 new_dataset["Close"][i]=data["Close"][i]
new_dataset.head()

In [ ]:
new_dataset.index=new_dataset.Date
new_dataset.drop("Date",axis='columns',inplace=True)
final_dataset=new_dataset.values
new_dataset.head()

In [ ]:
df=new_dataset.reset_index()['Close']
print(df)

In [ ]:
plt.plot(df, linewidth=0.7)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))
df=scaler.fit_transform(np.array(df).reshape(-1,1))
print(df)

In [ ]:
training_size=int(len(df)*0.65)
testing_size=len(df)-training_size
train_data,test_data=df[0:training_size,:],df[training_size:len(df),:1]

In [ ]:
import numpy
def create_dataset(dataset, time_step=1):
 dataX, dataY = [], []
 for i in range(len(dataset)-time_step-1):
  a = dataset[i:(i+time_step), 0] ###i=0, 0,1,2,3-----99 100
  dataX.append(a)
  dataY.append(dataset[i + time_step, 0])
 return numpy.array(dataX), numpy.array(dataY)
time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, ytest = create_dataset(test_data, time_step)
print(X_train.shape), print(y_train.shape)

In [ ]:
print(X_test.shape), print(ytest.shape)

In [ ]:
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

In [ ]:
model=Sequential()
model.add(LSTM(50,return_sequences=True,input_shape=(X_train.shape[1],1)))
model.add(Dropout(rate = 0.2))
model.add(LSTM(50,return_sequences=True))
model.add(Dropout(rate = 0.2))
model.add(LSTM(50,return_sequences=False))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,ytest),epochs=100,batch_size=32,verbose=1)

In [ ]:
train_predict=model.predict(X_train)
test_predict=model.predict(X_test)

In [ ]:
fig, axes = plt.subplots(nrows=1,ncols=2, figsize=(15, 6), squeeze=False)
fig.tight_layout(pad=5)
plt.style.use('fivethirtyeight')
axes[0][0].plot(y_train,color='b',linewidth=1)
axes[0][0].plot(train_predict,color='r',linewidth=1)
axes[0][0].set_title('Train Predictions')
axes[0][1].plot(ytest,color='b',linewidth=1)
axes[0][1].plot(test_predict,color='r',linewidth=1)
axes[0][1].set_title('Train Predictions')

In [ ]:
train_predict=scaler.inverse_transform(train_predict)
test_predict=scaler.inverse_transform(test_predict)

In [ ]:
math.sqrt(mean_squared_error(y_train,train_predict))

In [ ]:
mae = mean_absolute_error(ytest,test_predict)
mse = mean_squared_error(ytest,test_predict)
r2 = r2_score(ytest,test_predict)
rmse = math.sqrt(mean_squared_error(ytest,test_predict))
print("The model performance for testing set:")
for i in range(38):
 print("-", end='')
print("\n")
print('MAE is {}'.format(mae))
print('MSE is {}'.format(mse))
print('R2 is {}'.format(r2))
print('RMSE is {}'.format(rmse))

In [ ]:
look_back=100
trainPredictPlot = numpy.empty_like(df)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
testPredictPlot = numpy.empty_like(df)
testPredictPlot[:, :] = numpy.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(df)-1, :] = test_predict
plt.figure(figsize=(10, 5))
plt.plot(scaler.inverse_transform(df),linewidth=1)
plt.plot(trainPredictPlot,linewidth=1)
plt.plot(testPredictPlot,linewidth=1)
plt.legend(['Actual data set', 'Predicted results for training data set', 'Predicted results for test data set'])
plt.show()

In [ ]:
len(test_data)

In [ ]:
x_input=test_data[333:].reshape(1,-1)
x_input.shape

In [ ]:
temp_input=list(x_input)
temp_input=temp_input[0].tolist()

In [ ]:
from numpy import array
lst_output=[]
n_steps=380
i=1
while(i<=30):
 if(len(temp_input)>100):
  x_input=np.array(temp_input[1:])
  print("{} day input: \n\n{}".format(i,x_input))
  x_input=x_input.reshape(1,-1)
  x_input = x_input.reshape((1, n_steps, 1))
  yhat = model.predict(x_input, verbose=0)
  print("\n{} day output: {}\n\n\n".format(i,yhat))
  temp_input.extend(yhat[0].tolist())
  temp_input=temp_input[1:]
  lst_output.extend(yhat.tolist())
  i=i+1
else:
 x_input = x_input.reshape((1, n_steps,1))
 yhat = model.predict(x_input, verbose=0)
 print(yhat[0])
 temp_input.extend(yhat[0].tolist())
 print(len(temp_input))
 lst_output.extend(yhat.tolist())
 i=i+1

In [ ]:
day_new=np.arange(1,101)
day_pred=np.arange(101,132)
len(df)

In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(day_new, scaler.inverse_transform(df[1935:]), linewidth=2)
plt.plot(day_pred, scaler.inverse_transform(lst_output), linewidth=2)
plt.legend(['Historical Stock Price', 'Future Stock Prices'])
plt.show()